# Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening

### Zach Goodenow
10/11/17
<br>
CS 440

Still being developed, but you may get started on this when you are finished with Assignment 3.

# Table of Contents
* [Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening](#Assignment-4:-Negamax-with-Alpha-Beta-Pruning-and-Iterative-Deepening)
	* [Initial Code](#Initial-Code)
	* [Add moves counter](#Add-moves-counter)
	* [negamaxIDS](#negamaxIDS)
	* [negamaxIDSab](#negamaxIDSab)
	* [Grading](#Grading)
	* [Extra Credit](#Extra-Credit)


For this assignment, you will investigate the advantages of alpha-beta
pruning applied to Tic-Tac-Toe.  To do so, follow these steps.

### An interesting note about Tic Tac Toe

I found a website with a document of every single game of Tic Tac Toe. It gives the following numbers: 
- 255,168 unique games of Tic Tac Toe to be played. 
- Of these, 131,184 are won by the first player, 77,904 are won by the second player, and 46,080 are drawn.

## Initial Code

I replaced `negamax`, `TTT`, `opponent`, and `playGame` with my edits here...
<br><br>
Note that I was using a global boolean `debug` for printing info in various places all over my code.

In [1]:
debug = False

# Returns the best move for the current state of the game and the value associated
def negamax(game, depthLeft):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        if debug: print('Negamax Base Case will return ------------------- ', game.getUtility())
        return game.getUtility(), None  # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        # Negate it because it will come back in context of next player
        value, _ = negamax(game, depthLeft-1)
        if debug: print('\tDepth {}, Value {}, Move {}, Player {}'.format(depthLeft, value, move, game.playerLookAHead))
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
    return bestValue, bestMove

In [2]:
debug = False

class TTT(object):

    def __init__(self):
        self.board = [' ']*9
        self.player = 'X' # Player that is currently up
        if False: # Used to start a game at a different state.  Just change to true to use
            self.board = ['X', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'O']
            self.player = 'X'
        self.playerLookAHead = self.player # Next player to move
        self.movesExplored = 0 # Counter for number of moves explored
        self.winningValue = 1 # Winning move number when negamax searching

    # Returns the locations of X's, O's, or empty spots (c) for board
    def locations(self, c):
        return [i for i, mark in enumerate(self.board) if mark == c]

    # Returns empty spot locations of board
    def getMoves(self):
        moves = self.locations(' ')
        return moves

    # Returns information on the state of the board:
    #   1 = a player has won and is next up
    #   -1 = a player has won and not next up
    #   0 = game is a draw, no one wins
    #   None if the game is still being played
    def getUtility(self):
        # Get locations of X's
        whereX = self.locations('X')
        # Get locations of O's
        whereO = self.locations('O')
        # Lists of locations that would constitute a win
        wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
                [0, 3, 6], [1, 4, 7], [2, 5, 8],
                [0, 4, 8], [2, 4, 6]]
        # Boolean that checks if X has won
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        # Boolean that checks if O has won
        isOWon = any([all([wi in whereO for wi in w]) for w in wins])
        if isXWon:
            # If X has won and is the next player to move, return 1
            # If X has won but is not the next player to move, return -1
            return 1 if self.playerLookAHead is 'X' else -1
        elif isOWon:
            # If O has won and is the next player to move, return 1
            # If O has won but is not the next player to move, return -1
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            # If there is a draw (Cats game) return 0
            return 0
        else:
            # If the game is still being played, return None
            return None  ########################################################## CHANGED FROM -0.1

    # Returns True if game is over, False otherwise
    def isOver(self):
        return self.getUtility() is not None

    # Puts the current player's mark at the index passed.  DOESNT CHECK TO MAKE SURE THAT MOVE IS VALID...
    # Changes playerLookAhead because the next move is going to be made by the other player
    def makeMove(self, move):
        global debug
        self.board[move] = self.playerLookAHead
        if debug: print('MADE MOVE ', self.playerLookAHead, ' @ SPOT, ', move)
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'
        if debug: print('CHANGED NEXT UP TO: ', self.playerLookAHead)
        self.movesExplored += 1

    # Changes player.  Also changes playerLookAHead because the new player hasnt made move yet so they are next up
    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player
        global debug
        if debug: print('CHANGED PLAYER TO: ', self.player)

    # Removes mark from index passed and restores playerLookAHead b/c that move was taken back
    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'
        global debug
        if debug: print('UNMADE MOVE: ', move)

    # Returns the number of moves explored
    def getNumberMovesExplored(self):
        return self.movesExplored

    # Returns the number representing a win when negamax searching
    def getWinningValue(self):
        return self.winningValue

    def isEmpty(self):
        boo = True
        for i in self.board:
            if not i.isspace():
                boo = False
        return boo

    # To string
    def __str__(self):
        s = '{}|{}|{}\n-----\n{}|{}|{}\n-----\n{}|{}|{}'.format(*self.board)
        return s

Check that the following function `playGame` runs
correctly. Notice that we are using *negamax* to find the best move for
Player X, but Player O, the opponent, is using function *opponent*
that follows the silly strategy of playing in the first open position.

In [3]:
# A dumby opponent
def dummyTTTopp(game):
    return game.board.index(' ')

def playGame(game,opponent,depthLimit, algorithm = negamax):
    print(game)
    while not game.isOver():
        score,move = algorithm(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver(): # This section is playing for other player, opponent
            game.changePlayer()
            opponentMove = opponent(game)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)
            print(game)
            game.changePlayer()

In [4]:
game = TTT()
playGame(game,dummyTTTopp,10)

 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 


The above results are interesting b/c the third move for 'X' made by `negamax` was 4, not 6.  6 would have ended the game in a win but 4 also insures a win.  Why not just win?

## negamaxIDS 

<font color='red'>UPDATED Oct 4</font>

Write a new function named `negamaxIDS` that performs an iterative deepening negamax search.  Replace the first line in the `while` loop of `playGame` with

        score,move = negamaxIDS(game,depthLimit)
        
where `depthLimit` is now the maximum depth and multiple `negamax` searches are performed for depth limits of $1, 2, \ldots,$ maximum depth.

But, when should you stop?  Can you stop before readhing the depthLimit?  If not, there is no point to doing iterative deepening.

For Tic-Tac-Toe, we can stop as soon as a call to `negamax` returns a winning move.  This will have a value of 1 for Tic-Tac-Toe.  To keep our `negamaxIDS` function general, add a method called `getWinningValue` to the `TTT` class that just returns 1.  Then `negamaxIDS` can call `game.getWinningValue()` to determine the value of a winning move for this game.  If the maximum depth is reached and no winning move has been found, return the best move found over all depth limts.

<div class="alert alert-block alert-info">
*negamaxIDS* basically just runs *negamax* with *maxDepth* increasing.

For every depth, it updates *bestValue* and *bestMove*.  
<br>
**The key here** is that it early returns if value is game winning value.  Otherwise, it wouldn't be any more useful than just running *negamax* with maxDepth.  This is an attempt to optimize nodes searched because you could find the best move at a depth way before maxDepth.  However, if the best move will be found towards the end of an IDS search, there could actually be more nodes searched than just calling *negamax* with depth limit *maxDepth*.
<br><br>
Note that I was checking for values of *None* and *Inf* before *negamax* was updated to start *bestValue* at *None*

In [4]:
debug = False

# Performs an iterative deepening negamax search.  Returns best move for current state and value assosicated
def negamaxIDS(game, maxDepth):
    global debug
    bestValue = -float('infinity')
    bestMove = None
    # bestMove = game.getMoves()[0]
    for depth in range(1, maxDepth+1):
        value, move = negamax(game, depth)
        if value in {None, float('Inf')}: # removed-> , -float('Inf')
            if debug: print('***Depth {} found {} for move {}'.format(depth, value, move))
            continue
        if value == game.getWinningValue(): # Found winning move, so return it
            if debug: print('Found winning value at depth {} with move {}'.format(depth, move))
            return value, move
        if value > bestValue: # Found a new best move
            bestValue = value
            bestMove = move
        if debug:
            print('Depth {} has value {} for move {}'.format(depth, value, move))
            print('AND BestValue {} for BestMove {}'.format(bestValue, bestMove))
    return bestValue, bestMove

## negamaxIDSab

Now for the hardest part.  Make a new function `negamaxIDSab` by duplicating `negamaxIDS` and add the code to implement alpha-beta pruning.
<br><br>
**From Negamax notes:** <br>
Modify Negamax to perform alpha-beta cutoffs by making these changes:
- Two new arguments, `alpha` and `beta`, whose initial values are −∞ and ∞.
- In the for loop for trying moves, negate and swap the values of `alpha` and `beta`, and the returned value from recursive calls must be negated.
- Do early return if `bestScore` is greater than or equal to `beta`.
- Update `alpha` to maximum of `bestScore` and current `alpha`.

<div class="alert alert-block alert-info">
I applyed the bullet points above to *negamax* to make the function *negamaxab*.
<br><br>
*negamaxIDSab* : *negamaxab* <=> *negamaxIDS* : *negamax*
<br><br>
Dificulty was figuring out where/when to prune.  I was also switching and negating alpha and beta in the for loop when this should be occuring before exploring children nodes.
<br><br>
What I learned was that pruneing is helpful because if you find a move better than your opponents best move, it is garenteed to be a good move so you dont have to explore other options.  
<br>
**NOTE: I added an extra argument to *negamaxIDSab* called *skipOdds* ** that is set to False.  This was useful when making the extra credit game and is explained later.  This shouldnt be a problem when grading but if it is I will be happy to edit this addition.

In [5]:
debug = False

# negamaxIDS but with alpha beta pruning
def negamaxIDSab(game, maxDepth, alpha = -float('Inf'), beta = float('Inf'), skipOdds = False):
    global debug
    bestValue = -float('infinity')
    bestMove = None
    for depth in range(1, maxDepth + 1):
        if skipOdds and depth%2 != 0: # Used to improve defence
            continue
        value, move = negamaxab(game, depth)
        # Should I check for failure??????????????????????????
        # Why would negamax return infinity???????????????????
        if value in {None, float('Inf')}:  # removed-> , -float('Inf')
            if debug: print('***Depth {} found {} for move {}'.format(depth, value, move))
            continue
        if value == game.getWinningValue():  # Found winning move, so return it
            if debug: print('Found winning value at depth {} with move {}'.format(depth, move))
            return value, move
        if value > bestValue:  # Found a new best move
            bestValue = value
            bestMove = move
        if debug:
            print('Depth {} has value {} for move {}'.format(depth, value, move))
            print('AND BestValue {} for BestMove {}.  Nodes explored: {}.'.format(bestValue, bestMove, 
                                                                                  game.getNumberMovesExplored()))
    return bestValue, bestMove


# Negamax search but with alpha beta pruning
def negamaxab(game, depthLeft, alpha = -float('Inf'), beta = float('Inf')):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        # call to negamaxab knows the move
        return game.getUtility(), None
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    # Swap and negate alpha beta
    alpha, beta = -beta, -alpha
    if debug:
        print('{}\n^Game in ngmxAB -> Depth Left: {}, alpha: {}, beta: {}'.format(game, depthLeft, alpha, beta))
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)

        # Use depth-first search to find eventual utility value and back it up.
        value, _ = negamaxab(game, depthLeft - 1, alpha, beta)

        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue

        # Negate it because it will come back in context of next player
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move

        # Update alpha to maximum of bestScore and current alpha
        alpha = max(bestValue, alpha)

        if debug: print('\tmove {} has value: {}, alpha: {}, beta: {}, bestValue: {}, bestMove: {}'.format(move, 
                                                                                                     value, 
                                                                                                     alpha, 
                                                                                                     beta, 
                                                                                                     bestValue, 
                                                                                                     bestMove))
        # Do early return if bestScore is greater than or equal to beta
        if bestValue >= beta:
            if debug: print('\t*PRUNE FOUND* move {}. bestValue: {}, beta: {}, bestMove: {}'.format(move, 
                                                                                                    bestValue, 
                                                                                                    beta, 
                                                                                                    bestMove))
            return bestValue, bestMove
    return bestValue, bestMove

This is the basic outline I have learned for recursive search functions: <br>

**define function**
    1. Check base case and return if true
    2. Assign values that are equivalent for all child nodes will need
**loop through all children and...**
    3. Apply anything needed for traversing DOWN a child/node tree
**make recursive call**
    4. Check problem/useless values and take corrasponding action
    5. Apply anything needed for traversing UP a child/node tree

## playGames

Now duplicate the game playing loop so three complete tic-tac-toe games are played.  Call this new version `playGames`. For the first game, use `negamax`. For the second game, use `negamaxIDS`.  For the third game, use `negamaxIDSab`.  At the end of each game, print the number of X's in the final board, the number moves explored, the depth of the game which is the number of moves made by X and O, and the effective branching factor.  When you run `playGames` you should see the tic-tac-toe positions after each move and, after all games are done, a line for each game like the following lines, which were <font color='red'>UPDATED Oct 5</font>.

    negamax made 4 moves. 558334 moves explored for ebf(558334, 7) of 6.46
    negamaxIDS made 3 moves. 744695 moves explored for ebf(744695, 5) of 14.73
    negamaxIDSab made 3 moves 20804 moves explored for ebf(20804, 5) of 7.09

Your results may be different. 

The value of the depth is the total number of moves made by X and by O during the search.  You can calculate this by keeping a list of all board states, or by just counting the number of X's and O's in the final board.

Since `playGames` uses `ebf`, define an `ebf` function from A3. 
<br><br>
I actually used the code that Dr. Anderson posted on piazza for `ebf` to insure I have the same values.

In [6]:
def ebf(nNodes, depth, precision=0.01):
    if nNodes == 0:
        return 0

    def ebfRec(low, high):
        mid = (low + high) * 0.5
        if mid == 1:
            estimate = 1 + depth
        else:
            estimate = (1 - mid**(depth + 1)) / (1 - mid)
        if abs(estimate - nNodes) < precision:
            return mid
        if estimate > nNodes:
            return ebfRec(low, mid)
        else:
            return ebfRec(mid, high)

    return ebfRec(1, nNodes)

In [7]:
def playGames(opponent, depthLimit):
    ngmx_moves = ngmx_nodes = ngmx_depth = ngmx_ebf = None
    ngmxIDS_moves = ngmxIDS_nodes = ngmxIDS_depth = ngmxIDS_ebf = None
    ngmxIDSab_moves = ngmxIDSab_nodes = ngmxIDSab_depth = ngmxIDSab_ebf = None

    # negamax game
    print('negamax:')
    ngmx_game = TTT()
    playGame(ngmx_game, opponent, depthLimit, negamax)
    ngmx_moves = len(ngmx_game.locations('X'))
    ngmx_nodes = ngmx_game.getNumberMovesExplored()
    ngmx_depth = ngmx_moves + len(ngmx_game.locations('O'))
    ngmx_ebf = round(ebf(ngmx_nodes, ngmx_depth), 2)
    ngmx_report = 'negamax made {} moves. {} moves explored for ebf({}, {}) of {}'.format(ngmx_moves,
                                                                                          ngmx_nodes,
                                                                                          ngmx_nodes,
                                                                                          ngmx_depth,
                                                                                          ngmx_ebf)
    # negamaxIDS game
    print('\nnegamaxIDS:')
    ngmxIDS_game = TTT()
    playGame(ngmxIDS_game, opponent, depthLimit, negamaxIDS)
    ngmxIDS_moves = len(ngmxIDS_game.locations('X'))
    ngmxIDS_nodes = ngmxIDS_game.getNumberMovesExplored()
    ngmxIDS_depth = ngmxIDS_moves + len(ngmxIDS_game.locations('O'))
    ngmxIDS_ebf = round(ebf(ngmxIDS_nodes, ngmxIDS_depth), 2)
    ngmxIDS_report = 'negamaxIDS made {} moves. {} moves explored for ebf({}, {}) of {}'.format(ngmxIDS_moves,
                                                                                          ngmxIDS_nodes,
                                                                                          ngmxIDS_nodes,
                                                                                          ngmxIDS_depth,
                                                                                          ngmxIDS_ebf)
    print('\nnegamaxIDSab:')
    ngmxIDSab_game = TTT()
    playGame(ngmxIDSab_game, opponent, depthLimit, negamaxIDSab)
    ngmxIDSab_moves = len(ngmxIDSab_game.locations('X'))
    ngmxIDSab_nodes = ngmxIDSab_game.getNumberMovesExplored()
    ngmxIDSab_depth = ngmxIDSab_moves + len(ngmxIDSab_game.locations('O'))
    ngmxIDSab_ebf = round(ebf(ngmxIDSab_nodes, ngmxIDSab_depth), 2)
    ngmxIDSab_report = 'negamaxIDSab made {} moves. {} moves explored for ebf({}, {}) of {}'.format(ngmxIDSab_moves,
                                                                                                ngmxIDSab_nodes,
                                                                                                ngmxIDSab_nodes,
                                                                                                ngmxIDSab_depth,
                                                                                                ngmxIDSab_ebf)
    print(ngmx_report)
    print(ngmxIDS_report)
    print(ngmxIDSab_report)

Here are some example results.

In [9]:
playGames(dummyTTTopp, 10)

negamax:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 

negamaxIDS:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 

negamaxIDSab:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 
negam

Output from example results updated October 8, 8:00pm:

negamax made 4 moves. 558334 moves explored for ebf(558334, 7) of 6.46<br>
negamaxIDS made 3 moves. 23338 moves explored for ebf(23338, 5) of 7.26<br>
negamaxIDSab made 3 moves 6053 moves explored for ebf(6053, 5) of 5.48

## User Input Opponent

I found it very helpful to play against my search functions to further validate that they are making optimal moves.  The following code is used to play against any of the search functions.

In [8]:
# Keyboard input opponent
def UI(game):
    validMoves = game.getMoves()
    moveUI = int(input('Your Move...\n'))
    while moveUI not in validMoves:
        print('\nNOT A VALID MOVE! >:D\n')
        print('Pick from one of the following ---> ', validMoves)
        moveUI = int(input(''))
    return moveUI


def playTTT_1vComp(game,depthLimit,opponent = UI, algorithm = negamaxIDSab):
    comp = 'X'
    resp = str.lower(input('Would you like to go first? (Y(es) or N(o))\n'))
    if 'n' in resp:
        game.makeMove(4)
        comp = 'X'
    print(game)
    print('COMPUTER IS', comp)
    while not game.isOver():
        game.changePlayer()
        opponentMove = opponent(game)
        game.makeMove(opponentMove)
        print('Player', game.player, 'to', opponentMove)
        print(game)
        game.changePlayer()
        # Computer's turn
        if game.isOver():
            break
        score, move = algorithm(game, depthLimit)
        if move == None:
            print('move is None so picking first spot')
            move = game.getMoves()[0]
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score', score)
        print(game)
    if game.getUtility() == 0: print('Cats game... Its a draw!')
    elif len(game.locations('X')) > len(game.locations('O')): print('HAHAHAHA I WIN!!!!!!')
    else: print('No way you won!?!?!?')
    print('Game Utility: ', game.getUtility())
    print('Nodes explored during this game: ', game.getNumberMovesExplored())

**UNCOMMENT ONE OF THE PLAY GAME LINES AND RUN TO PLAY**...I like the second one

In [9]:
TTTgame = TTT()
# playGame(TTTgame,UI,10)
# playTTT_1vComp(TTTgame, 10)

## Grading

As always, download and extract from [A4grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A4grader.tar)

In [12]:
# from A4mysolution import *

In [10]:
%run -i A4grader.py


Testing negamax starting from ['O', 'X', ' ', 'O', ' ', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of 1

--- 10/10 points. negamax correctly explored 124 states.

Testing negamax starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of -1 and move of 5

Testing negamaxIDS with max depth of 5, starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamaxIDS correctly returns value of -1 and move of 5

Testing negamaxIDSab starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 20/20 points. negamaxIDSab correctly returns value of -1 and move of 5

Testing playGame with opponent that always plays in highest numbered position.
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 8
X| | 
-----
 | | 
-----
 | |O
Player X to 2 for score 1
X| |X
-----
 | | 
-----
 | |O
Player O to 7
X| |X
-----
 | | 
-----
 |O|O
Player X to 1 for 

## Extra Credit: Apply Connect Four

Earn one extra credit point for each of the following.

  - Implement another game and repeat the above steps.

  - Implement a random move chooser as the opponent (Player O) and determine how many times Player X can win against this opponent as an average over multiple games.

The game that I implemented was Connect Four.  In Connect Four, players take turns dropping colored discs from the top into a seven-column, six-row vertically suspended grid.  The pieces fall straight down, occupying the next available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own discs. I read online that Connect Four is a solved game, meaning that the first player can always win by playing the right moves.  Hence, a proper implementation will result in an artificial intelligence that will always win...
<br><br>
The valid moves for a C4 game are columns of the board that have not been filled yet.  I modeled the C4 class after the TTT class given.  One problem here is that `getUtility` in TTT only returns a value if the game is won, lost, or a draw.  This is very unhelpful for C4 because the game is much more complex and the search functions will not get deep enough to find an optimal move.  For this reason I put in A LOT of code to caculate the game utility.  Most are obvious for what they do but the others have comments above to explain their usage.

Notes:


**IMPORTANT TO KNOW**<br>
For my C4 game, an iterative deepening search is not the best option.  This is because at depth 1, the IDS will find what it thinks is the best move but it is actually the worst move.  Here is a good example of this behavior: 
<br><br>
$$
\begin{array}{ccc}
 &0&1&2&3&4&5&6\\
0& & & & & & & \\
1& & & & & & & \\
2& & &O&O& & & \\
3& & &O&X& & & \\
4& &X&X&O& & & \\
5&O&X&X&X&O& & \\
\end{array}
$$

An IDS `negamaxab` search looks through all depths and finds that move (Row: 3, Col: 1) has the highest game utility value.  This move has a high offensive value because it creates 2 groups of 3 in a row.  But it gives an opportunity for `O` to win the game.  Why does `negamaxIDSab` choose this move?  It’s because at depth limit 1, `negamaxIDSab` finds the highest value to be move (Row: 3, Col: 1).  And for the rest of the depth limits, the best value is never higher since player ‘O’ has an overall better game utility.  

Hence, the opportunity to prune this move is prevented by an IDS of negamaxab. `negamaxIDSab` ignorantly chooses not to take the next opponent move into account.  This method works when the game utility only returns a value when a game is over.  But in my C4 game, the game utility returns a value for every instance of the game.

For this reason, **I added an extra argument to `negamaxIDSab` called `skipOdds`** that is set to False.  This is so `negamaxIDSab` will always call `negamaxab` with an even depth limit and thus will always take the next opponent move into account. I think there are better ways to counter this problem by manipulating Connect Four Get Utility itself.

In [11]:
debug = False

class C4(object):

    def __init__(self):
        self.board = [[' ' for x in range(7)] for y in range(6)] # Make a 7x6 board
        self.player = 'X' # Player that is currently up
        if False: # Used to start a game at a different state.  Just change to true to use
            self.board = [[' ', ' ', ' ', ' ', ' ', ' ', ' '],
                          [' ', ' ', ' ', ' ', ' ', ' ', ' '],
                          [' ', ' ', 'O', 'O', ' ', ' ', ' '],
                          [' ', ' ', 'O', 'X', ' ', ' ', ' '],
                          [' ', 'X', 'X', 'O', ' ', ' ', ' '],
                          ['O', 'X', 'X', 'X', 'O', ' ', ' ']] # This example motivated checking singleMoveWins > 1 in utility
            self.player = 'X'
        self.playerLookAHead = self.player # Next player to move
        self.movesExplored = 0 # Counter for number of moves explored
        self.winningValue = 1000 # Winning move number when negamax searching -- 4 in a row

    # Returns actual row object, not pointer
    def getRow(self, rowNum):
        if rowNum > 5 or rowNum < 0:
            raise ValueError(str(rowNum) + ' not a valid getRow request!')
        else:
            return self.board[rowNum]

    # Returns new object, copy of that column
    def getCol(self, colNum):
        if colNum > 6 or colNum < 0:
            raise ValueError(str(colNum) + ' not a valid getCol request!')
        else:
            boardColumns = [list(x) for x in zip(*self.board)]
            return boardColumns[colNum]
        
    # Returns the number of pieces in the board
    def locations(self, c):
        numberP = 0
        for row in self.board:
            numberP += row.count(c)
        return numberP
    
    # Returns columns that are not full
    def getMoves(self):
        moves = []
        index = 0
        for spot in self.board[0]:
            if spot is ' ':
                moves.append(index)
            index += 1
        return moves

    # Returns information on the state of the board
    def getUtility(self):
        won, winner = self.isWon()
        if winner is 'X':
            # If X has won and is the next player to move, return 1000
            # If X has won but is not the next player to move, return -1000
            return 1000 if self.playerLookAHead is 'X' else -1000
        elif winner is 'O':
            # If O has won and is the next player to move, return 1000
            # If O has won but is not the next player to move, return -1000
            return 1000 if self.playerLookAHead is 'O' else -1000
        elif self.isFull():
            # If there is a draw (Cats game) return 0
            return 0
        else:
            # If the game is still being played, find bestIAR X - bestIAR O and apply singleMoveWins for X & O
            # ADDED SINGLE MOVE WINS.  A single move win adds 10 to respective bestIAR
            bestX = self.bestIAR_All('X')
            if debug: print('Best IAR X:', bestX)
            singleMoveWinsX = self.singleMoveWinCount('X')
            if debug: print('Single moves to win X:', singleMoveWinsX)
            # if 5 > singleMoveWinsX > 1: # There is 2 or more moves X can make to win.
                # return 500 if self.playerLookAHead is 'X' else -500
                # singleMoveWinsX = 5
            bestX += singleMoveWinsX * 10

            bestO = self.bestIAR_All('O')
            if debug: print('Best IAR O:', bestO)
            singleMoveWinsO = self.singleMoveWinCount('O')
            if debug: print('Single moves to win O:', singleMoveWinsO)
            # if 5 > singleMoveWinsO > 1: # There is 2 or more moves O can make to win.
                # return 500 if self.playerLookAHead is 'O' else -500
                # singleMoveWinsO = 5
            bestO += singleMoveWinsO * 10

            if singleMoveWinsX > 1 and singleMoveWinsO < 2:  # There is 2 or more moves X can make to win.
                return 500 if self.playerLookAHead is 'X' else -500
            if singleMoveWinsO > 1 and singleMoveWinsX < 2:  # There is 2 or more moves X can make to win.
                return 500 if self.playerLookAHead is 'O' else -500

            best = bestX - bestO
            return best if self.playerLookAHead is 'X' else (best*-1)

    # Returns True if game is over, False otherwise
    def isOver(self):
        winBoo,_ = self.isWon()
        return winBoo

    # Puts the current player's mark at the column index passed.  DOESNT CHECK TO MAKE SURE THAT MOVE IS VALID...
    # Changes playerLookAhead because the next move is going to be made by the other player
    def makeMove(self, move):
        if move not in self.getMoves():
            raise ValueError(str(move) + ' not a valid column to make move!')
        global debug
        # Get row index to put piece
        currCol = self.getCol(move)
        blanks = [i for i, mark in enumerate(currCol) if mark == ' ']
        rowIndex = max(blanks)
        # Place piece in column
        self.board[rowIndex][move] = self.playerLookAHead
        # if debug: print('MADE MOVE {} @ ROW {} COL {}'.format(self.playerLookAHead, rowIndex, move))
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'
        # if debug: print('CHANGED NEXT UP TO:', self.playerLookAHead)
        self.movesExplored += 1

    # Changes player.  Also changes playerLookAHead because the new player hasnt made move yet so they are next up
    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    # Sets player and player look ahead.  Idk if this is the best way to do this...
    def setPlayer(self, piece):
        self.player = piece
        self.playerLookAHead = self.player

    # Removes mark from index passed and restores playerLookAHead b/c that move was taken back
    def unmakeMove(self, move):
        global debug
        # Get row index to remove piece
        currCol = self.getCol(move)
        if ' ' not in currCol:
            rowIndex = 0
        else:
            blanks = [i for i, mark in enumerate(currCol) if mark == ' ']
            # if debug: print('unmakeMove blanks:', blanks)
            rowIndex = max(blanks) + 1 if blanks is not None else 0

        if rowIndex > 5:
            raise ValueError(str(move) + ' not a valid column to remove piece from!')
        # Remove piece in column
        self.board[rowIndex][move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    # Returns the number of moves explored
    def getNumberMovesExplored(self):
        return self.movesExplored

    # Returns the number representing a win when negamax searching
    def getWinningValue(self):
        return self.winningValue

    def isEmpty(self):
        for row in self.board:
            if not ''.join(row).isspace():
                return False
        return True

    def isFull(self):
        for row in self.board:
            if ' ' in row:
                return False
        return True

    # Returns true and winner if won, false and ' ' otherwise
    def isWon(self):
        hBoo, hWinner = self.isHorzWin()
        if hBoo:
            return hBoo, hWinner
        vBoo, vWinner = self.isVertWin()
        if vBoo:
            return vBoo, vWinner
        duBoo, duWinner = self.isDiagUpWin()
        if duBoo:
            return duBoo, duWinner
        ddBoo, ddWinner = self.isDiagDownWin()
        if ddBoo:
            return ddBoo, ddWinner
        return False, ' '

    # Returns true and piece of 4IAR if list has 4 pieces in a row (IAR), false and ' ' otherwise
    def has4IAR(self, group):
        lookFor = ' '
        cons = 0
        for spot in group:
            if spot is lookFor:
                cons += 1
            else:
                lookFor = spot
                cons = 1
            if cons is 4 and lookFor is not ' ':
                return True, lookFor
        return False, ' '

    # Returns true and winner if there is 4 in a row in any horizontal position, false and ' ' otherwise
    def isHorzWin(self):
        for row in self.board:
            boo, winner = self.has4IAR(row)
            if boo:
                return True, winner
        return False, ' '

    # Returns true and winner if there is 4 in a row in any vertical position, false and ' ' otherwise
    def isVertWin(self):
        boardColumns = [list(x) for x in zip(*self.board)]
        for col in boardColumns:
            boo, winner = self.has4IAR(col)
            if boo:
                return True, winner
        return False, ' '

    # Returns true and winner if there is 4 in a row in any diagonal up position, false and ' ' otherwise
    def isDiagUpWin(self):
        for du in self.getDiagUp():
            boo, winner = self.has4IAR(du)
            if boo:
                return True, winner
        return False, ' '

    # Returns true and winner if there is 4 in a row in any diagonal down position, false and ' ' otherwise
    def isDiagDownWin(self):
        for dd in self.getDiagDown():
            boo, winner = self.has4IAR(dd)
            if boo:
                return True, winner
        return False, ' '

    # returns diagonal up lists of board. I HATE THIS FUNCTION BUT IT WORKS FOR RIGHT NOW. CHANGE LATER!!
    def getDiagUp(self):
        up = []
        grab = []
        rowStarti = 3
        rowi = 3
        coli = 0
        while rowStarti < 6:
            grab.append(self.board[rowi][coli])
            if rowi is 0:
                rowStarti += 1
                rowi = rowStarti
                coli = 0
                up.append(grab)
                grab = []
            else:
                rowi -= 1
                coli += 1
        grab = []
        colStarti = 1
        rowi = 5
        coli = 1
        while colStarti < 4:
            grab.append(self.board[rowi][coli])
            if coli is 6:
                colStarti += 1
                coli = colStarti
                rowi = 5
                up.append(grab)
                grab = []
            else:
                rowi -= 1
                coli += 1
        return up

    # returns diagonal down lists of board. I HATE THIS FUNCTION BUT IT WORKS FOR RIGHT NOW. CHANGE LATER!!
    def getDiagDown(self): 
        down = []
        grab = []
        rowStarti = 2
        rowi = 2
        coli = 0
        while rowStarti >= 0:
            grab.append(self.board[rowi][coli])
            if rowi is 5:
                rowStarti -= 1
                rowi = rowStarti
                coli = 0
                down.append(grab)
                grab = []
            else:
                rowi += 1
                coli += 1
        grab = []
        colStarti = 1
        rowi = 0
        coli = 1
        while colStarti < 4:
            grab.append(self.board[rowi][coli])
            if coli is 6:
                colStarti += 1
                coli = colStarti
                rowi = 0
                down.append(grab)
                grab = []
            else:
                rowi += 1
                coli += 1
        return down

    #############################################################################################################
    # Best in a row stuff

    # Returns the best overall best in a row for given piece on given board.  Wont ever be more than 4
    def bestIAR_All(self, piece):
        if debug: print('IN bestIAR_ALL FOR', piece)
        hBIAR = 0
        for row in self.board:
            hBIAR = max(hBIAR, self.bestIAR(row, piece))
        if debug: print('\tBest Horizontal:', hBIAR)
        hBIAR = hBIAR ** 2

        vBIAR = 0
        boardColumns = [list(x) for x in zip(*self.board)]
        for col in boardColumns:
            vBIAR = max(vBIAR, self.bestIAR(col, piece))
        if debug: print('\tBest Vertical:', vBIAR)
        vBIAR = vBIAR ** 2

        duBIAR = 0
        for u in self.getDiagUp():
            duBIAR = max(duBIAR, self.bestIAR(u, piece))
        if debug: print('\tBest Diag Up:', duBIAR)
        duBIAR = duBIAR ** 2

        ddBIAR = 0
        for d in self.getDiagDown():
            ddBIAR = max(ddBIAR, self.bestIAR(d, piece))
        if debug: print('\tBest Diag Down:', ddBIAR)
        ddBIAR = ddBIAR ** 2

        # overAllBest = max(hBIAR, vBIAR, duBIAR, ddBIAR)
        return hBIAR+vBIAR+duBIAR+ddBIAR #overAllBest if overAllBest < 4 else 4

    # Returns highest number of pieces ('piece') that could combine for a win for group (row, col, or diag)
    # EX: [' ', ' ', 'X', 'X', 'O', ' ', ' '] - Best X: 2, Best O: 0
    # EX: ['X', 'X', ' ', 'X', 'O', ' ', ' '] - Best X: 3, Best O: 0
    def bestIAR(self, group, piece):
        best = 0
        chunks = self.groupToChunks(group, piece)
        for chunk in chunks:
            best = max(best, chunk.count(piece))
        return best

    # Function gets chunks of potential win sets for piece
    # EX: group = [' ', ' ', 'X', 'X', 'O', ' ', ' '], piece = 'X' ==> [[' ', ' ', 'X', 'X']]
    # EX: group = [' ', 'X', 'X', 'O', 'O', ' ', ' '], piece = 'X' ==> []
    # EX: group = [' ', 'X', 'X', 'O', 'O', ' ', ' '], piece = 'O' ==> [['O', 'O', ' ', ' ']]
    def groupToChunks(self, group, piece):
        # Get indexs of piece or empty
        indexs = []
        for i, p in enumerate(group):
            if p in [piece, ' ']:
                indexs.append(i)

        # Get split indexs
        spl = [0] + [i for i in range(1, len(indexs)) if indexs[i] - indexs[i - 1] > 1] + [None]
        # print('spl:', spl)
        # List those index
        fin = [indexs[b:e] for (b, e) in [(spl[i - 1], spl[i]) for i in range(1, len(spl))]]
        # print('fin:', fin)
        # Indexs to elements from group
        grab = []
        for lst in fin:
            put = []
            for elm in lst:
                put.append(group[elm])
            # print('put:', put)
            if piece in put and len(put) >= 4: grab.append(put)
        return grab

    # Returns a count of single moves that would win for piece
    def singleMoveWinCount(self, piece):
        # print('IN SINGLE MOVES WIN COUNT WITH UTILITY {}'.format(self.getUtility()))
        # Save state of game
        PLAsave = self.playerLookAHead
        Psave = self.player
        self.setPlayer(piece)

        # Count single move wins
        singleMoveWinCounter = 0
        for m in self.getMoves():
            self.makeMove(m)
            # print('Move {} made utility {}'.format(m, self.getUtility()))
            if self.isWon()[0]:
                if debug: print('\tFOUND {}s WINNING MOVE {} IN SMWC'.format(piece, m))
                singleMoveWinCounter += 1
            self.unmakeMove(m)

        self.playerLookAHead = PLAsave
        self.player = Psave
        return singleMoveWinCounter

    # To string
    def __str__(self):
        top = '_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _\n'
        bottom = '\n-----------------------------'
        boardData = []
        rowIndex = 0
        for row in self.board:
            boardData.append(str(rowIndex) + ' {} | {} | {} | {} | {} | {} | {} |'.format(*row))
            rowIndex += 1
        printBoard = top + '\n|---+---+---+---+---+---+---|\n'.join(boardData) + bottom
        return printBoard

In [12]:
# Check that its working
checkC4 = C4()
print(checkC4)

_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
3   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
4   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
5   |   |   |   |   |   |   |
-----------------------------


In [13]:
for row in range(6):
    for col in range(7):
        checkC4.makeMove(col)
print(checkC4)

_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0 O | X | O | X | O | X | O |
|---+---+---+---+---+---+---|
1 X | O | X | O | X | O | X |
|---+---+---+---+---+---+---|
2 O | X | O | X | O | X | O |
|---+---+---+---+---+---+---|
3 X | O | X | O | X | O | X |
|---+---+---+---+---+---+---|
4 O | X | O | X | O | X | O |
|---+---+---+---+---+---+---|
5 X | O | X | O | X | O | X |
-----------------------------


## playGames

Same as playGames in TTT but had to change the game type.  Could have just edited playGames but I didnt want to mess with the grader.  
<br>
I also added a random choice move opponent for C4

In [14]:
import random

# A random move C4 opponent.  D is so it can be used as a dummy algorithm too
def randomC4opp(game, d = 10):
    moves = game.getMoves()
    return random.choice(moves)

# A dumby opponent
def dummyC4opp(game):
    moves = game.getMoves()
    return moves[0]

def playGameC4(game,opponent,depthLimit, algorithm = negamaxIDS, printOut = True):
    if printOut: print(game)
    while not game.isOver():
        score,move = algorithm(game,depthLimit)
        if move == None :
            move = dummyC4opp(game)
        game.makeMove(move)
        if printOut: print('Player', game.player, 'to', move, 'for score' ,score)
        if printOut: print(game)
        if not game.isOver(): # This section is playing for other player, opponent
            game.changePlayer()
            opponentMove = opponent(game)
            game.makeMove(opponentMove)
            if printOut: print('Player', game.player, 'to', opponentMove)
            if printOut: print(game)
            game.changePlayer()

def playGamesC4(opponent, depthLimit):
    ngmx_moves = ngmx_nodes = ngmx_depth = ngmx_ebf = None
    ngmxIDS_moves = ngmxIDS_nodes = ngmxIDS_depth = ngmxIDS_ebf = None
    ngmxIDSab_moves = ngmxIDSab_nodes = ngmxIDSab_depth = ngmxIDSab_ebf = None

    # negamax game
    print('negamax:')
    ngmx_game = C4()
    playGameC4(ngmx_game, opponent, depthLimit, negamax)
    ngmx_moves = ngmx_game.locations('X')
    ngmx_nodes = ngmx_game.getNumberMovesExplored()
    ngmx_depth = ngmx_moves + ngmx_game.locations('O')
    ngmx_ebf = round(ebf(ngmx_nodes, ngmx_depth), 2)
    ngmx_report = 'negamax made {} moves. {} moves explored for ebf({}, {}) of {}'.format(ngmx_moves,
                                                                                          ngmx_nodes,
                                                                                          ngmx_nodes,
                                                                                          ngmx_depth,
                                                                                          ngmx_ebf)

    # negamaxIDS game
    print('\nnegamaxIDS:')
    ngmxIDS_game = C4()
    playGameC4(ngmxIDS_game, opponent, depthLimit, negamaxIDS)
    ngmxIDS_moves = ngmxIDS_game.locations('X')
    ngmxIDS_nodes = ngmxIDS_game.getNumberMovesExplored()
    ngmxIDS_depth = ngmxIDS_moves + ngmxIDS_game.locations('O')
    ngmxIDS_ebf = round(ebf(ngmxIDS_nodes, ngmxIDS_depth), 2)
    ngmxIDS_report = 'negamaxIDS made {} moves. {} moves explored for ebf({}, {}) of {}'.format(ngmxIDS_moves,
                                                                                          ngmxIDS_nodes,
                                                                                          ngmxIDS_nodes,
                                                                                          ngmxIDS_depth,
                                                                                          ngmxIDS_ebf)

    print('\nnegamaxIDSab:')
    ngmxIDSab_game = C4()
    playGameC4(ngmxIDSab_game, opponent, depthLimit, negamaxIDSab)
    ngmxIDSab_moves = ngmxIDSab_game.locations('X')
    ngmxIDSab_nodes = ngmxIDSab_game.getNumberMovesExplored()
    ngmxIDSab_depth = ngmxIDSab_moves + ngmxIDSab_game.locations('O')
    ngmxIDSab_ebf = round(ebf(ngmxIDSab_nodes, ngmxIDSab_depth), 2)
    ngmxIDSab_report = 'negamaxIDSab made {} moves. {} moves explored for ebf({}, {}) of {}'.format(ngmxIDSab_moves,
                                                                                                ngmxIDSab_nodes,
                                                                                                ngmxIDSab_nodes,
                                                                                                ngmxIDSab_depth,
                                                                                                ngmxIDSab_ebf)

    print(ngmx_report)
    print(ngmxIDS_report)
    print(ngmxIDSab_report)

In [18]:
playGamesC4(dummyC4opp, 4)

negamax:
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
3   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
4   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
5   |   |   |   |   |   |   |
-----------------------------
Player X to 0 for score -2
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
3   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
4   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
5 X |   |   |   |   |   |   |
-----------------------------
Player O to 0
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2   |   |   |   |   

Player X to 4 for score 19
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
3   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
4   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
5 O |   |   | X | X |   |   |
-----------------------------
Player O to 0
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
3   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
4 O |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
5 O |   |   | X | X |   |   |
-----------------------------
Player X to 2 for score 1000
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|


Implement a random move chooser as the opponent (Player O) and determine how many times Player X can win against this opponent as an average over multiple games.

In [19]:
agame = C4()
playGameC4(agame, randomC4opp, 4, negamaxab, False)
print(agame)

_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2 X |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
3 X |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
4 X |   |   |   | O |   |   |
|---+---+---+---+---+---+---|
5 X |   | O |   | X | O | O |
-----------------------------


In [20]:
print(agame.isWon())

(True, 'X')


Play any number of games with random moves and see how many times it wins.  I have yet to see a loss.

In [15]:
numWins = 0
howManyGames = 100

for numGames in range(howManyGames):
    aC4game = C4()
    playGameC4(aC4game, randomC4opp, 4, negamaxab, False)
    won,winner = aC4game.isWon()
    if winner is 'X': 
        numWins += 1
    print('Game number {} result \n{}\n'.format(numGames, aC4game))
          
# print('X won {} out of {} games for average {}'.format(numWins, howManyGames, (numWins/howManyGames)))
print('X won {} out of {} games for average {}'.format(numWins, howManyGames, (numWins/howManyGames)))

Game number 0 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
3   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
4 X | X | X | X |   |   |   |
|---+---+---+---+---+---+---|
5 X | O | O | X | O | O | O |
-----------------------------

Game number 1 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
3   |   |   |   |   |   | X |
|---+---+---+---+---+---+---|
4 O |   |   | O |   |   | O |
|---+---+---+---+---+---+---|
5 X | X | X | X |   |   | O |
-----------------------------

Game number 2 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1 O |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2 

Game number 20 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   | O |
|---+---+---+---+---+---+---|
2 X |   |   |   |   |   | O |
|---+---+---+---+---+---+---|
3 X | X | X |   |   |   | X |
|---+---+---+---+---+---+---|
4 X | O | X |   |   |   | O |
|---+---+---+---+---+---+---|
5 X | O | O | X |   | O | O |
-----------------------------

Game number 21 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2 X |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
3 X |   |   | O |   |   |   |
|---+---+---+---+---+---+---|
4 X |   | X | X |   |   |   |
|---+---+---+---+---+---+---|
5 X | O | O | X | O | O | O |
-----------------------------

Game number 22 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|

Game number 40 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   | O |   | X |   |
|---+---+---+---+---+---+---|
1 X | O |   | O |   | O |   |
|---+---+---+---+---+---+---|
2 O | X |   | X |   | X | X |
|---+---+---+---+---+---+---|
3 X | X |   | X | O | X | X |
|---+---+---+---+---+---+---|
4 O | X |   | O | X | O | O |
|---+---+---+---+---+---+---|
5 X | O |   | X | O | O | O |
-----------------------------

Game number 41 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
3   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
4 O |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
5 X | X | X | X | O |   | O |
-----------------------------

Game number 42 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   | X |   |   |   |   |   |
|---+---+---+---+---+---+---|

Game number 60 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   | O |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   | O |   |   |   |
|---+---+---+---+---+---+---|
2 X | X | X | X |   |   |   |
|---+---+---+---+---+---+---|
3 X | O | O | O | X |   |   |
|---+---+---+---+---+---+---|
4 O | X | X | X | O |   |   |
|---+---+---+---+---+---+---|
5 X | O | O | X | O |   |   |
-----------------------------

Game number 61 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2   |   | O | X |   |   |   |
|---+---+---+---+---+---+---|
3 O |   | X | X |   | O |   |
|---+---+---+---+---+---+---|
4 X | X | O | X | X | O |   |
|---+---+---+---+---+---+---|
5 X | X | O | O | X | O | O |
-----------------------------

Game number 62 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|

Game number 80 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0 O |   |   | X |   |   |   |
|---+---+---+---+---+---+---|
1 O |   |   | X |   | O |   |
|---+---+---+---+---+---+---|
2 O |   |   | X |   | X |   |
|---+---+---+---+---+---+---|
3 X |   | O | X |   | O |   |
|---+---+---+---+---+---+---|
4 X |   | X | O |   | O |   |
|---+---+---+---+---+---+---|
5 X | O | X | X |   | O |   |
-----------------------------

Game number 81 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
2   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
3   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
4 O |   |   | O |   |   |   |
|---+---+---+---+---+---+---|
5 X | X | X | X |   |   | O |
-----------------------------

Game number 82 result 
_ 0 _ 1 _ 2 _ 3 _ 4 _ 5 _ 6 _
0   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|
1   |   |   |   |   |   |   |
|---+---+---+---+---+---+---|

In [22]:
print('X won {} out of {} games for average {}'.format(numWins, howManyGames, (numWins/howManyGames)))

X won 10 out of 10 games for average 1.0


## User Input For Connect Four

The following code uses user input to play with any negamax search.  

In [27]:
# A dummy algorithm
def dummyAlg(game, d):
    moves = game.getMoves()
    return 0, moves[0]

def playC4_1vComp(game,depthLimit,opponent = UI, algorithm = dummyAlg, skipO = False):
    comp = 'X'
    resp = str.lower(input('Would you like to go first? (Y(es) or N(o))\n'))
    if 'n' in resp:
        game.makeMove(3)
        comp = 'X'
    print(game)
    print('COMPUTER IS', comp)
    while not game.isOver():
        game.changePlayer()
        opponentMove = opponent(game)
        game.makeMove(opponentMove)
        print('Player', game.player, 'to', opponentMove)
        print(game)
        game.changePlayer()
        # Computer's turn
        if game.isOver():
            break
        if skipO:
            score, move = algorithm(game, depthLimit, alpha = -float('Inf'), beta = float('Inf'), skipOdds = skipO)
        else:
            score, move = algorithm(game, depthLimit)
        if move == None:
            print('move is None so picking first spot')
            move = game.getMoves()[0]
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score', score)
        print(game)
    won, winner = game.isWon()
    if not won:
        print('Cats game... Its a draw!')
    elif winner is comp:
        print('HAHAHAHA I WIN!!!!!!')
    else:
        print('No way you won!?!?!?')
    print('Player {} won!'.format(winner))
    print('Nodes explored during this game: ', game.getNumberMovesExplored())

Yet to win.  Try to beat it and let me know if you do!!! email: zachgood@rams.colostate.edu

In [30]:
# Uncomment and run to play!!!  I have yet to win with this line of code...
neverGoingToWin = C4()
# playC4_1vComp(neverGoingToWin, 4, opponent = UI, algorithm = negamaxIDSab, skipO = True)

I won here a few times but still not easy!

In [29]:
# Uncomment and run to play!!!  Ive only won once here
C4gameUI = C4()
# playC4_1vComp(C4gameUI, 4, opponent = UI, algorithm = negamaxIDSab)

## Alg vs. Alg C4

In [25]:
def playC4_CvC(game, depthLimit, alg1 = dummyAlg, alg2 = negamaxIDSab):
    print(game)
    while not game.isOver():
        score, move = alg1(game, depthLimit)
        if move == None:
            move = dummyC4opp(game)
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score', score)
        print(game)
        if not game.isOver():  # This section is playing for other player, opponent
            game.changePlayer()
            opponentScore, opponentMove = alg2(game, depthLimit)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove, 'for score', opponentScore)
            print(game)
            game.changePlayer()
    _, winner = game.isWon()
    print('Player {} won!'.format(winner))
    print('Nodes explored during this game: ', game.getNumberMovesExplored())

Ive played around with this function to learn about some flaws!

In [ ]:
# playC4_CvC(C4game, 4, negamaxIDS, negamaxIDSab) # This is an interesting example